In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
# needed whenever working with spark dataframes
from pyspark.sql import *
!rm -rf metastore_db/
sqlContext = SQLContext(sc)

In [ ]:
!wget https://ibm.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv

In [ ]:
# using inferschema is very handy, but adds a second pass over the data

df = sqlContext.read.format('com.databricks.spark.csv')\
                .option('header', 'true')\
                .option('inferschema', 'true')\
                .option('mode', 'DROPMALFORMED')\
                .load('f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv')

In [ ]:
df.count()

In [ ]:
df.printSchema()

In [ ]:
df.show(5)

In [ ]:
df.select('car', 'mpg').show(5)

In [ ]:
df.filter(df['mpg'] < 18).show(5)

In [ ]:
# the filter can be just like a SQL where clause
df.filter('mpg < 18').show(5)

In [ ]:
# dataframes are immutable -- withColumn creates a new one
df.withColumn('wtTon', df['wt'] * 0.45).show(6)

In [ ]:
df.groupby(['cyl'])\
.agg({"wt": "AVG"})\
.show(5)

In [ ]:
car_counts = df.groupby(['cyl'])\
.agg({"*": "count"})\
.sort("count(1)", ascending=False)\
.show(5)

In [ ]:
df.createOrReplaceTempView("cars")

# SQL statements can be run by using the sql method
highgearcars = sqlContext.sql("SELECT gear FROM cars WHERE cyl >= 4 AND cyl <= 9")
highgearcars.show(6)

## Exercise 1:

Step 1: Create a list of tuples ("john", 23), ("mr. bean", 56), ("bill clinton", 70)

Step 2: Convert the list to a data frame with two columns

Step 3: Filter the people with age < 40

Step 4: Get the names ONLY of people smaller than 40

Step 5: Register the dataframe as a SQL table

Step 6: Select the names only of people whose age < 40

Some hints: http://spark.apache.org/docs/latest/sql-programming-guide.html#generic-loadsave-functions

## Exercise 2:

Step 1: Create a list of tuple ("john", "male"), ("hilary", "female")

Step 2: Join with the data from the previos exercise and output all males older than 50

Hints: https://databricks.com/blog/2015/02/17/introducing-dataframes-in-spark-for-large-scale-data-science.html

In [ ]:
sc.stop()